<a href="https://colab.research.google.com/github/beysach/beysach/blob/main/audio_to_srt_khmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnxruntime-gpu==1.17
!pip install kfa
!pip install pydub SpeechRecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.8/396.8 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.7 MB/s

In [ ]:
import speech_recognition as sr

# Function to transcribe audio file
def transcribe_audio(input_audio_file):
    recognizer = sr.Recognizer()

    # Load audio file
    with sr.AudioFile(input_audio_file) as source:
        audio_data = recognizer.record(source)

    # Transcribe audio
    try:
        text = recognizer.recognize_google(audio_data,  language="km-KH")
        # text = recognizer.recognize_google(audio_data)
        print(text)
        return text
    except sr.UnknownValueError:
        return "Speech recognition could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"


transcribe_data = ""
# Main function
def transcribe_audio_to_text(input_audio_file, output_file):
    transcribed_text = transcribe_audio(input_audio_file)
    transcribe_data = transcribed_text

    if transcribed_text:
        with open(output_file, "w", encoding="utf-8") as f:
          f.write(transcribed_text)
    else:
        print("Transcription failed.")


transcribe_audio_to_text("/content/audio.wav","transcribed_output.txt")



ខ្ញុំសាកល្បងធ្វើខ្មែរសត្វថៃថ៍ខ្លួនឯងដែលប្រើប្រាស់នៅតែជារបស់កូនខ្មែរបាទបោះកូនខ្មែរមែនគឺយើងប្រើ kfa ដែលបង្កើតដោយ


In [ ]:
import onnxruntime as rt
rt.get_device()




'CPU'

In [ ]:
import torch
from kfa import align, create_session
import librosa

with open("/content/transcribed_output.txt") as infile:
    text = infile.read()

y, sr = librosa.load("/content/audio.wav", sr=16000, mono=True)
session = create_session(providers=["CUDAExecutionProvider"])
segments = []
for alignment in align(y, sr, text, session=session):
  print(alignment)
  segments.append(alignment)

def generate_srt_from_tuples(tuples, output_srt_file):
    with open(output_srt_file, "w", encoding="utf-8") as f:
        for idx, tup in enumerate(tuples, start=1):
            start_time = format_time(tup[1] * 1000)  # Convert seconds to milliseconds
            end_time = format_time(tup[2] * 1000)  # Convert seconds to milliseconds
            text = tup[0]
            f.write(f"{idx}\n")
            f.write(f"{start_time} --> {end_time}\n")
            f.write(f"{text}\n\n")

def format_time(milliseconds):
    seconds = milliseconds / 1000
    minutes = seconds / 60
    hours = minutes / 60
    return f"{int(hours):02d}:{int(minutes)%60:02d}:{int(seconds)%60:02d},{int(milliseconds)%1000:03d}"

generate_srt_from_tuples(segments, 'test.srt')

/root/.cache/kfa/wav2vec2-km-base-1500.onnx.tmp: 100%|██████████| 360M/360M [00:21<00:00, 18.0MiB/s]
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
100%|██████████| 1/1 [00:02<00:00,  2.10s/it]

('ខ្ញុំ', 0, 0.2204675, 0.0200425, 0.16034, 0.5682877719622671)
('សាកល្បង', 0.2204675, 0.7415725, 0.2204675, 0.7014875, 0.7521439039458832)
('ធ្វើ', 0.7415725, 1.2225925, 0.7415725, 1.1424225000000001, 0.6560929918428883)
('ខ្មែរ', 1.2225925, 1.7436975, 1.2225925, 1.723655, 0.7872157087735832)
('Subtitle', 1.7436975, 2.4051, 1.7436975, 2.365015, 0.659711366710854)
('ខ្លួន', 2.4051, 2.5854825, 2.4051, 2.5253550000000002, 0.5076594140233889)
('ឯង', 2.5854825, 2.8259925, 2.5854825, 2.7658650000000002, 0.7101422395143244)
('ដែល', 2.8259925, 2.96629, 2.8259925, 2.9462475, 0.6474638506770134)
('ប្រើប្រាស់', 2.96629, 3.6276925, 2.96629, 3.5876075000000003, 0.6522938221122229)
('នៅ', 3.6276925, 3.76799, 3.6276925, 3.7479475000000004, 0.5526270043725768)
('បច្ចេកវិទ្យា', 3.76799, 4.3492225, 3.76799, 4.32918, 0.5205945931881745)
('របស់', 4.3492225, 4.5696900000000005, 4.3492225, 4.5496475, 0.4698001851727895)
('កូន', 4.5696900000000005, 4.8502849999999995, 4.5696900000000005, 4.7901575, 0.467193